In [1]:
from transformers import AutoTokenizer
from optimum.onnxruntime import (
    ORTModelForSeq2SeqLM,
)

model_id = "SEBIS/code_trans_t5_base_commit_generation_transfer_learning_finetune"

# model = ORTModelForSeq2SeqLM.from_pretrained(model_id, export=True)
# print(model)

/home/kevin/commit-t5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model.save_pretrained("ct-base-commits")
model = ORTModelForSeq2SeqLM.from_pretrained("ct-base-commits")

In [3]:
from optimum.onnxruntime import  OptimizationConfig, ORTOptimizer, ORTModelForSeq2SeqLM

save_dir = "ct-base-commits-optimized"

# Create the optimizer
optimizer = ORTOptimizer.from_pretrained(model)

# Define the optimization strategy by creating the appropriate configuration
optimization_config = OptimizationConfig(
    optimization_level=2,
    enable_transformers_specific_optimizations=True,
    optimize_for_gpu=False,
)

# Optimize the model
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
optimized_model = ORTModelForSeq2SeqLM.from_pretrained(save_dir)
tokens = tokenizer("This is a sample input", return_tensors="pt")
outputs = optimized_model.generate(**tokens)

/home/kevin/commit-t5/venv/lib/python3.10/site-packages/optimum/onnxruntime/configuration.py:735: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
symbolic shape inference disabled or failed.
Configuration saved in ct-base-commits-optimized/ort_config.json
Optimized model saved at: ct-base-commits-optimized (external data format: False; saved all tensor to one file: True)
Generation config file not found, using a generation config created from the model config.
/home/kevin/commit-t5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [3]:
from optimum.onnxruntime import (
    AutoOptimizationConfig, ORTOptimizer
)

save_dir = "ct-base-commits-onnx"

optimizer = ORTOptimizer.from_pretrained(model)
optimization_config = AutoOptimizationConfig.O3()

# Optimize the model
optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)

/home/kevin/commit-t5/venv/lib/python3.10/site-packages/optimum/onnxruntime/configuration.py:735: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
symbolic shape inference disabled or failed.
Configuration saved in ct-base-commits-onnx/ort_config.json
Optimized model saved at: ct-base-commits-onnx (external data format: False; saved all tensor to one file: True)


PosixPath('ct-base-commits-onnx')

In [4]:
model_onnx = ORTModelForSeq2SeqLM.from_pretrained("ct-base-commits-onnx")

Generation config file not found, using a generation config created from the model config.


In [5]:
from optimum.pipelines import pipeline
onnx_seq2seq = pipeline("text2text-generation", model=model_onnx, accelerator="ort")

In [9]:
%%time

input_text = """
diff --git a/README.md b/README.md
index 9b077d6..c9ee84c 100644
--- a/README.md
+++ b/README.md
@@ -6,7 +6,7 @@ React is a JavaScript library for building user interfaces.
 * **Efficient:** React minimizes interactions with the DOM by using a mock representation of the DOM.
 * **Flexible:** React works with the libraries and frameworks that you already know.
 
-[Learn how to use React in your own project.](http://facebook.github.io/docs/getting-started.html)
+[Learn how to use React in your own project.](http://facebook.github.io/react/docs/getting-started.html)
 
 ## Examples
 
@@ -41,7 +41,7 @@ The fastest way to get started is to serve JavaScript from the CDN:
 <script src="http://fb.me/JSXTransformer-0.3.0.js"></script>
 ```
 
-We've also built a [starter kit](#) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
+We've also built a [starter kit](http://facebook.github.io/react/downloads/react-0.3.0.zip) which might be useful if this is your first time using React. It includes a webpage with an example of using React with live code.
 
 If you'd like to use [bower](http://bower.io), it's as easy as:
"""
onnx_seq2seq(input_text)

CPU times: user 8.73 s, sys: 4.46 ms, total: 8.74 s
Wall time: 691 ms


[{'generated_text': 'Add a download of React in README . md'}]